In [1]:
# base on https://huggingface.co/docs/transformers/training

In [2]:
!rm -rf ojt_bert
!git clone https://github.com/wzwzeyal/ojt_bert

Cloning into 'ojt_bert'...
remote: Enumerating objects: 223, done.
remote: Counting objects: 100% (223/223), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 223 (delta 101), reused 178 (delta 59), pack-reused 0
Receiving objects: 100% (223/223), 9.29 MiB | 15.42 MiB/s, done.
Resolving deltas: 100% (101/101), done.


In [3]:
!pip install datasets transformers

     |████████████████████████████████| 311 kB 4.3 MB/s 
     |████████████████████████████████| 3.4 MB 43.5 MB/s 
     |████████████████████████████████| 67 kB 3.7 MB/s 
     |████████████████████████████████| 133 kB 56.2 MB/s 
     |████████████████████████████████| 1.1 MB 47.8 MB/s 
     |████████████████████████████████| 243 kB 51.5 MB/s 
     |████████████████████████████████| 596 kB 40.3 MB/s 
     |████████████████████████████████| 3.3 MB 40.8 MB/s 
     |████████████████████████████████| 895 kB 50.6 MB/s 
     |████████████████████████████████| 271 kB 37.1 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 144 kB 55.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
from datasets import load_dataset, DatasetDict, Dataset



# Configuration

In [5]:
MODEL_CKPT = "onlplab/alephbert-base"
TEXT_COLUMN_NAME = "comment"
LABEL_COLUMN_NAME = "label"
HUGGINGFACE_LABEL_COLUMN_NAME = "labels"
NUM_LABELS = 3
TRAIN_BATCH_SIZE = 8
EVAL_BATCH_SIZE = 8
NUM_EPOCS = 3

# Preparing the datasets


In [6]:
import pandas as pd

train_df = pd.read_csv('./ojt_bert/data/for_sentiment/train_token_df.gz')
test_df = pd.read_csv('./ojt_bert/data/for_sentiment/val_token_df.gz')
raw_datasets = DatasetDict()
raw_datasets["train"] = Dataset.from_pandas(train_df[[TEXT_COLUMN_NAME, LABEL_COLUMN_NAME]])
raw_datasets["test"] = Dataset.from_pandas(test_df[[TEXT_COLUMN_NAME, LABEL_COLUMN_NAME]])
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['comment', 'label'],
        num_rows: 4616
    })
    test: Dataset({
        features: ['comment', 'label'],
        num_rows: 681
    })
})

In [7]:
# raw_datasets = load_dataset("imdb")

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_CKPT)

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/532k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [9]:
def tokenize_function(examples):
    return tokenizer(examples[TEXT_COLUMN_NAME], padding="max_length", truncation=True)

In [10]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['comment', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4616
    })
    test: Dataset({
        features: ['comment', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 681
    })
})

In [11]:
tokenized_datasets = tokenized_datasets.remove_columns([TEXT_COLUMN_NAME])
tokenized_datasets = tokenized_datasets.rename_column(LABEL_COLUMN_NAME, HUGGINGFACE_LABEL_COLUMN_NAME)
tokenized_datasets.set_format("torch")

# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]

In [12]:
from torch.utils.data import DataLoader

In [13]:
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=TRAIN_BATCH_SIZE)
eval_dataloader = DataLoader(eval_dataset, batch_size=EVAL_BATCH_SIZE)

In [14]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(MODEL_CKPT, num_labels=NUM_LABELS)

Downloading:   0%|          | 0.00/481M [00:00<?, ?B/s]

Some weights of the model checkpoint at onlplab/alephbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at onlplab/alephbert-base

In [15]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [16]:
from transformers import get_scheduler

num_epochs = NUM_EPOCS
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [17]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
from tqdm.auto import tqdm

import numpy as np
from datasets import load_metric

metric= load_metric("accuracy", "f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

progress_bar = tqdm(range(num_training_steps))


for epoch in range(num_epochs):
  model.train()
  for batch in train_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)
  
  model.eval()
  for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch[HUGGINGFACE_LABEL_COLUMN_NAME])

  metric.compute()

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

  0%|          | 0/1731 [00:00<?, ?it/s]